In [ ]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
import mysql.connector
import pandas as pd

import json

with open('config.json') as f:
    config = json.load(f)


cnx = mysql.connector.connect(user=config['user'],
                              password=config['password'],
                              host=config['host'],
                              database=config['database'])

cursor = cnx.cursor()

# Example query
query = ("SELECT * FROM PH_Temp_Diet")

cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

# Create a pandas DataFrame
data = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


cursor.close()
cnx.close()

In [6]:
df3 =data
df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data.drop_duplicates()

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,06.09.2017,1,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
93,6.03,36.1,07.09.2017,1,0.0,0.0,0.0,0.0,0.0,180.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
186,5.81,36.4,08.09.2017,1,0.0,0.0,0.0,0.0,0.0,250.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
279,6.10,36.6,09.09.2017,1,0.0,0.0,110.0,500.0,0.0,100.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
372,6.06,35.9,10.09.2017,1,0.0,0.0,100.0,0.0,0.0,150.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19251,5.75,36.0,01.04.2018,1,222.0,0.0,0.0,0.0,0.0,21.0,...,0.0,250.0,0.0,0.0,0.0,126.0,1400.0,0.0,0.0,0.0
19344,5.37,36.1,02.04.2018,1,193.0,0.0,0.0,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,158.0,1050.0,0.0,0.0,0.0
19437,5.82,36.5,03.04.2018,1,140.0,0.0,78.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19530,5.03,36.1,04.04.2018,1,214.0,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0


In [14]:
# Define a function to recommend food items based on pH level
def recommend_foods(ph_level, df):
    # Calculate the mean pH for each food item
    food_phs = df.iloc[:, 5:].mean()
    # Determine which food items are associated with low or high pH levels
    low_phs = food_phs[food_phs < ph_level].index.tolist()
    high_phs = food_phs[food_phs > ph_level].index.tolist()
    
    # Return the recommended food items
    if ph_level < 7:
        return high_phs
    else:
        return low_phs

In [15]:
recommend_foods(6.8,joined_data)

['Banana',
 'Beer',
 'Braised Potatoes',
 'Bread',
 'Cake',
 'Cheese',
 'Cookie',
 'Croissant',
 'Curd',
 'Curd Pie',
 'Dried apricots',
 'Fried potatoes',
 'Fruit tea',
 'Ice cream',
 'Latte',
 'Lemon water',
 'Liquid',
 'Mashed potatoes',
 'Mayonnaise salad',
 'Meat',
 'Meat pilaf',
 'Meatless pilaf',
 'Melon',
 'Mineral water (Borjomi)',
 'Mineral water (Esentuki - 4)',
 'Mineral water (Prolom)',
 'Morse',
 'Noodles with shrimp',
 'Oatmeal',
 'Orange juice',
 'Patty with cabbage',
 'Peach',
 'Pear',
 'Pizza with meat',
 'Plum',
 'Potato dumplings',
 'Pumpkin seeds',
 'Red wine',
 'Scrambled eggs',
 'Sushi',
 'Tangerines',
 'Tea(black/green)',
 'The nuts',
 'Vegetable salad',
 'Water',
 'Watermelon',
 'White wine',
 'Zucchini Fritters']

In [17]:
import pickle
with open('recommend_diet.pkl', 'wb') as f:
    pickle.dump(joined_data, f)